In [9]:
from rich import print
import pandas as pd
import numpy as np
import math
import ast
from dataclasses import dataclass
from collections import defaultdict
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

In [23]:
class Participant:
    def __init__(self, id, university, interests, preferred_role, friend_registration, preferred_team_size, availability,
                 programming_skills, interest_in_challenges, experience, languages_ordered, maturity, Tryhard, Rookie, Learner, Portfolio):
        self.id = id
        self.university = university
        self.interests = interests
        self.preferred_role = preferred_role
        self.friend_registration = friend_registration
        self.preferred_team_size = preferred_team_size
        self.availability = availability
        self.programming_skills = programming_skills
        self.interest_in_challenges = interest_in_challenges
        self.experience = experience
        self.languages_ordered = languages_ordered
        self.maturity = maturity
        self.Tryhard = Tryhard
        self.Rookie = Rookie
        self.Learner = Learner
        self.Portfolio = Portfolio

In [11]:
INF = 999999

In [12]:
data_path = "data/datathon_participants.json"


In [13]:
def dist_university(university_1, university_2) :
    if university_1 == university_2:
        return 0 
    else:
        return INF

In [43]:
def dist_friends(friend_registration_1, friend_registration_2, id_1, id_2):
    if id_1 in friend_registration_2 and id_2 in friend_registration_1:
        return 0
    elif id_1 in friend_registration_2 or id_2 in friend_registration_1:
        return 1
    return 10

In [14]:
#No usar
def dist_age_year_of_study(age_1, age_2, year_1, year_2, age_weight, year_weigth) -> float:
    age_distance = abs(age_1 - age_2)
    year_distance = abs(year_1 - year_2)
    return (age_weight * age_distance + year_weigth * year_distance) / (age_distance + year_distance)


In [15]:
def dist_one_hot_encoding(x, y):
    return sum([1 if x_i != y_i else 0 for x_i, y_i in zip(x, y)])

In [16]:
def dist_programming_skills(skills_1, skills_2) -> float:
    avg_level_1 = sum(skills_1.values()) / len(skills_1) if skills_1 else 0
    
    avg_level_2 = sum(skills_2.values()) / len(skills_2) if skills_2 else 0
    
    distance_avg_levels = abs(avg_level_1 - avg_level_2)
    
    all_skills = set(skills_1.keys()).union(set(skills_2.keys()))
    
    distance_union = 1 / len(all_skills) if all_skills else INF 
    
    return distance_avg_levels + distance_union


In [17]:
#No usar
def dist_experience(exp_1, exp_2, hackathons_1, hackathons_2, experience_weights) -> float:
    exp_distance = abs(exp_1 - exp_2)
    hackathon_distance = abs(np.log1p(hackathons_1) - np.log1p(hackathons_2))
    total_distance = experience_weights[0] * exp_distance + experience_weights[1] * hackathon_distance
    return total_distance

In [18]:
def dist_preferred_role(role_1, role_2):
    if role_1 == role_2:
        return 1.0  

    if role_1 == "Don't know" or role_2 == "Don't know":
        return 1 / 0.2  

    if role_1 == "Don't care" or role_2 == "Don't care":
        return 0.5  

    return 0.0

In [19]:
#No usar
def euclidean_distance(vector_1, vector_2):
    squared_diff_sum = sum((x - y) ** 2 for x, y in zip(vector_1, vector_2))
    
    return math.sqrt(squared_diff_sum)

In [20]:
def dist_language(languages_1, languages_2):
    if not languages_1 or not languages_2:  
        return 0
    
    common_languages = set(languages_1).intersection(set(languages_2))
    
    if not common_languages:  
        return INF

    distance = 0
    for i, lang_1 in enumerate(languages_1):
        for j, lang_2 in enumerate(languages_2):
            if lang_1 == lang_2:
                d = abs(i - j)
                return d + math.exp(i+j) - 1

In [24]:
df = pd.read_csv('clean_data_actualitzat.csv')


In [26]:
participants_dict = {}

for _, row in df.iterrows():
    programming_skills = ast.literal_eval(row['programming_skills'])  
    interests = ast.literal_eval(row['interests']) 
    friend_registration = ast.literal_eval(row['friend_registration'])  
    availability = ast.literal_eval(row['availability'])  
    interest_in_challenges = ast.literal_eval(row['interest_in_challenges'])  
    languages_ordered = ast.literal_eval(row['languages_ordered'])  

    
    participant = Participant(
        id=row['id'],
        university=row['university'],
        interests=interests,  
        preferred_role=row['preferred_role'],
        friend_registration=friend_registration,  
        preferred_team_size=row['preferred_team_size'],
        availability=availability,  
        programming_skills=programming_skills,  
        interest_in_challenges=interest_in_challenges,  
        experience=row['experience'],
        languages_ordered=languages_ordered,  
        maturity=row['maturity'],
        Tryhard=row['Tryhard'],
        Rookie=row['Rookie'],
        Learner=row['Learner'],
        Portfolio=row['Portfolio']
    )
    
    participants_dict[row['id']] = participant

In [45]:
weights = {
    'university': 0.2,
    'interests': 0.2,
    'preferred_role': 0.3,
    'availability': 0.1,
    'programming_skils': 0.1,
    'interests_in_challenges': 0.05,
    'languages': 0.05,
    'experience': 0.05,
    'maturity': 0.15,
    'profile':0.2,
    'friends' : 1

}


In [46]:
def combined_distance(participant1: Participant, participant2: Participant, weigths: dict[str,float]):
    d_university = dist_university(participant1.university, participant2.university)*weigths['university']
    d_interests = dist_one_hot_encoding(participant1.interests, participant2. interests)*weigths['interests']
    d_preferred_role = dist_preferred_role(participant1.preferred_role, participant2. preferred_role)*weigths['preferred_role']
    d_availability = dist_one_hot_encoding(participant1.availability, participant2. availability)*weigths['availability']
    d_programming_skills = dist_programming_skills(participant1.programming_skills, participant2.programming_skills)*weigths['programming_skils']
    d_interests_in_challenges = dist_one_hot_encoding(participant1.interest_in_challenges, participant2.interest_in_challenges)*weigths['interests_in_challenges']
    d_languages = dist_language(participant1.languages_ordered, participant2.languages_ordered)*weigths['languages']
    d_experience = abs(participant1.experience - participant2.experience)*weigths['experience']
    d_maturity = abs(participant1.maturity - participant2.maturity)*weigths['maturity']
    d_profile = (abs(participant1.Tryhard - participant2.Tryhard) + abs(participant1.Rookie - participant2.Rookie) + abs(participant1.Learner - participant2.Learner) +  abs(participant1.Portfolio - participant2.Portfolio))*weights['profile']
    d_friend = dist_friends(participant1.friend_registration, participant2.friend_registration, participant1.id, participant2.id) * weights['friends']
    total_distance = (
        d_university +
        d_interests +
        d_preferred_role +
        d_availability +
        d_programming_skills +
        d_interests_in_challenges +
        d_languages + 
        d_experience +
        d_maturity +
        d_profile
    )
    return total_distance

In [47]:
combined_distance(participants_dict['2ebad15c-c0ef-4c04-ba98-c5d98403a90c'],participants_dict['2ebad15c-c0ef-4c04-ba98-c5d98403a90c'],weights)

0.3111111111111111